### __CE-406771:__ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; _Natural Language Processing, Fall 23._ <br>
### __Homework #1:__ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; _Text Exploration. <br>_
### __Personal Information:__ &nbsp; _Mohammad M. Gharaguzlo, 401206836, [My mail](moh.gharaguzlo13@sharif.edu)._ <br>

## _Introduction:_ <br>
As stated in the proposed homework document, we are to obtain a text file sized around one to five megabytes and conduct some exploratory tasks on it. <br>
The means as how to obtain the aforementioned text file and it's subject is left to be determined by ourselves.  <br>
As such, I chose to extract _The World Health Organization_'s fact sheets found [here](https://www.who.int/news-room/fact-sheets) which conatins concise but valueable information regarding commonly found diseases and disorders around the globe.
<br>
<div style="text-align: center;">
   <img src="pictures\who_fact_root.png" style="width: 60%;">
   <p style="text-align: center">World Health Organization's fact sheets</p>
</div>
<br>
I intend to download every webpage that inclueds a fact sheet (there should be around 200 of them), then extract the main article containing the information I need.

### _Installing Prerequisites and Dependencies:_ <br>
Alongside the common frameworks regarding language processing and text manipulation, we're going to need some libraries which can help us in downloading and parsing web pages. Personally, I used conda to installs the packages.
1. [Requests](https://requests.readthedocs.io/en/latest/)
2. [Beautifulsoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
3. [NLTK](https://www.nltk.org/)

In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install nltk

### _Importing Libraries:_ <br>
Assuming you've got all the dependencies installed on your machine, next step is to import them. We're gonna need [Requests](https://requests.readthedocs.io/en/latest/) to download the pages. [Beautifulsoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) will be used to parse html files and we'll utilize [regular expressions](https://docs.python.org/3/library/re.html) to search for certain patterns. Needless to say [NLTK](https://www.nltk.org/) will come in handy when it comes to anything NLP related.

In [1]:
from re import search, sub, findall
from time import sleep
from os import scandir
from requests import get
from bs4 import BeautifulSoup
from string import punctuation
import nltk

WHO_URL = "https://www.who.int"
HTML_PATH = "./html_files"

### _Downloading Appropriate Pages:_ <br>
The following lines of code will download [this page](https://www.who.int/news-room/fact-sheets) then iterates over every URL in it. obviously there exists many links redirecting to different parts of the WHO website but we're only intersted in fact sheets, so only if the obtained URL contains "fact-sheets" sub domain we'll downloand them.<br>
<div style="text-align: center;">
   <img src="pictures\who_links.png" style="width: 60%;">
   <p style="text-align: center">URLs leading to fact sheets</p>
</div>
<br>

From Requests we'll utilize "get" function to obtain html files, then using Beautifulsoup's "find_all" method we extract every link. With help of python's "search" function from re library we easily differentiate between desired and undesired URLs.<br>
__Make sure to create a directory named "html_files" before running this section (by default, it should be included in my homework files) !!__

In [ ]:
root = get("".join([WHO_URL, "/news-room/fact-sheets"]))
with open("root.html", "w") as root_descriptor:
    root_descriptor.write(root.text)

with open("root.html", "r") as file_descriptor:
    contents = file_descriptor.read()
soup = BeautifulSoup(contents, "html.parser")

for url in soup.find_all('a'):
    if search(r"/news-room/fact-sheets/", url.get('href')):
        html_raw    = get("".join([WHO_URL, url.get('href')])).text
        html_parsed = BeautifulSoup(html_raw, "html.parser")
        html_name   = html_parsed.find('title').text.strip().replace("/", "")
        print(html_name)
        with open("/".join([HTML_PATH, html_name]), 'w') as file_descriptor:
            file_descriptor.write(html_raw)
        sleep(2)

### _Extracting Content:_ <br>
If you've succesfully ran the previous scripts, your html directory should include about 200 files named in accordance to thier contents. 
<br>
<div style="text-align: center;">
   <img src="pictures\who_files.png" style="width: 60%;">
   <p style="text-align: center">Downloaded fact sheets</p>
</div>
<br>
Each html file contains a fact sheet regarding a certain disorder. The main section of each file is it's "article".
Utilizing Beautifulsoup's "find" method we can extract the article. We'll do this for every article and store the resulting contents in a text file named "aggregated.txt".
<br>
<div style="text-align: center;">
   <img src="pictures\who_article.png" style="width: 60%;">
   <p  style="text-align: center">Contents of a html file</p>
</div>
<br>
Before we continue to the next section we need to fix something, the acquired text has a large number of words sticked together! take a look this picture.
<br>
<div style="text-align: center;">
   <img src="pictures\who_sticked.png" style="width: 60%;">
   <p style="text-align: center">Sticked words</p>
</div>
<br>
There are more than 500 occurences of this phrase which needs to be handled. Again using regex we can split these words. The final result can be seen in "aggregated" file. Every article in encased between two "#" signs and dollar sign marks the article topic.
<br>
<div style="text-align: center;">
   <img src="pictures\who_agg.png" style="width: 60%;">
   <p style="text-align: center">Gathered content</p>
</div>
<br>

In [3]:
# Gathring articles in a single object

html_names = [html_file.name for html_file in scandir(HTML_PATH)]
html_names.sort()

content = "#\n"
for name in html_names:
    with open("/".join([HTML_PATH, name])) as file_descriptor:
        html_raw    = file_descriptor.read()

    html_parsed = BeautifulSoup(html_raw, "html.parser")
    core_text = html_parsed.find('article').text.replace("\n", " ").strip()
    content = "".join([content, "$"+name+"\n", core_text, "\n#\n"])
    print("Added {} article".format(name))

Added Abortion article
Added Abuse of older people article
Added Adolescent and young adult health article
Added Adolescent pregnancy article
Added Ageing and health article
Added Alcohol article
Added Ambient (outdoor) air pollution article
Added Anaemia article
Added Animal bites article
Added Antibiotic resistance article
Added Antimicrobial resistance article
Added Anxiety disorders article
Added Arsenic article
Added Asbestos_ elimination of asbestos-related diseases article
Added Assistive technology article
Added Asthma article
Added Autism article
Added Bacterial vaginosis article
Added Biodiversity and Health article
Added Blindness and vision impairment article
Added Blood safety and availability article
Added Botulism article
Added Breast cancer article
Added Brucellosis article
Added Burns article
Added Buruli ulcer article
Added Campylobacter article
Added Cancer article
Added Cardiovascular diseases (CVDs) article
Added Cervical cancer article
Added Chagas disease (also k

In [4]:
# Removing sticked words and writing the final result into a file

print("Sticked words sample: ", findall(r"[A-Z][a-z]+[A-Z][a-z]+\s", content)[:15])
print("Number of sticked words before removing them: ", len(findall(r"[A-Z][a-z]+[A-Z][a-z]+\s", content)))
while len(findall(r"[A-Z][a-z]+[A-Z][a-z]+\s", content)) > 0:
    content = sub(r"([A-Z][a-z]+)([A-Z][a-z]+\s)", r"\1 \2",content)    
print("Number of sticked words after removing them: ",len(findall(r"[A-Z][a-z]+[A-Z][a-z]+\s", content)))


with open("aggregated.txt", "w") as file_descriptor:
    file_descriptor.write(content)

Sticked words sample:  ['OverviewThe ', 'ConsequencesAbuse ', 'PreventionMany ', 'OverviewSurvival ', 'TuberculosisTuberculosis ', 'NutritionMany ', 'OverviewAdolescent ', 'ReferencesSully ', 'OverviewPeople ', 'OverviewAlcohol ', 'OverviewOutdoor ', 'PollutantsParticulate ', 'OverviewAnaemia ', 'CausesAnaemia ', 'ReferencesGlobal ']
Number of sticked words before removing them:  501
Number of sticked words after removing them:  0


### _Preprocessing:_ <br>
Now that we have the data we need, next step is conduct some preprocessing operations on the articles. These include removing stop words, unifying alphabet letters(lowering) and so on.<br>
Since I'm going to summarize the articles, it is necessary to remove stop words. Because the summarization procedure assigns a certain weight to each word based on it's frequency and based on the fact that the stop words has high frequency, having them in the sentence can really mess things up. Same applies to punctuation signs.<br>
However when it comes to stemming or lemmatization, their usefulness is quentionable(at least in this task which I'm implementing) since it may impact the readability of summarized piece in a negative way.


In [17]:
stop_words = nltk.corpus.stopwords.words('english')


with open("aggregated.txt", "r") as file_descriptor:
    content = file_descriptor.read()

content = content.split("#")

for subject, article in map(lambda section: section.strip().split("\n"), content[1:-1]):
    sent_tokenized  = nltk.sent_tokenize(article)
    word_tokenized  = nltk.word_tokenize(article)
    sent_lowered    = list(map(lambda sent: sent.lower(), sent_tokenized))
    word_lowered    = list(map(lambda word: word.lower(), word_tokenized))
    word_removed_sw = list(filter(lambda word: True if word not in stop_words else False, word_lowered))
    word_removed_pn = list(filter(lambda word: True if word not in punctuation else False, word_removed_sw))
    freq_in_article = nltk.probability.FreqDist(word_removed_pn)

    print("Article subject: {}".format(subject))
    print("Word frequency before preprocessing: {}".format(nltk.probability.FreqDist(word_tokenized).most_common(10)))
    print("Word frequency before preprocessing: {}".format(nltk.probability.FreqDist(word_removed_pn).most_common(10)))
    print("=======================================================================================")


Article subject: $Abortion
Word frequency before preprocessing: [(',', 123), ('and', 82), ('.', 76), ('of', 75), ('abortion', 61), ('the', 55), ('to', 45), ('(', 31), (')', 31), ('care', 29)]
Word frequency before preprocessing: [('abortion', 63), ('care', 29), ('health', 27), ('unsafe', 17), ('abortions', 16), ('women', 10), ('information', 9), ('quality', 9), ('safe', 9), ('countries', 9)]
Article subject: $Abuse of older people
Word frequency before preprocessing: [(',', 66), ('of', 58), ('the', 35), ('and', 34), ('abuse', 31), ('.', 30), ('older', 23), ('(', 21), (')', 21), ('people', 19)]
Word frequency before preprocessing: [('abuse', 37), ('older', 23), ('people', 19), ('elder', 11), ('health', 9), ('2', 7), ('financial', 6), ('1', 6), ('settings', 6), ('5', 6)]
Article subject: $Adolescent and young adult health
Word frequency before preprocessing: [('and', 87), (',', 87), ('of', 68), ('the', 56), ('in', 55), ('.', 51), ('adolescents', 40), ('to', 33), ('health', 25), ('years',

### _Summarizing:_ <br>
Note that the preprocessing part is also duplicated in this section, previously I just wanted to show how prepocessing works but here we utilize it.<br>
Now, given the format of my data set I thought it would be a good idea to summarize each article into 3 sentences. It works by giving each words a factor of importance with respect to it's normalized frequency in the article.<br>
Then for every sentece we calcualate the summation of it's  word's weights. Finally we choose the 3 top most sentences regarding it's score and write them into the summarized file.

In [22]:
summarized_content = ""
for subject, article in map(lambda section: section.strip().split("\n"), content[1:-1]):
    sent_tokenized  = nltk.sent_tokenize(article)
    word_tokenized  = nltk.word_tokenize(article)
    sent_lowered    = list(map(lambda sent: sent.lower(), sent_tokenized))
    word_lowered    = list(map(lambda word: word.lower(), word_tokenized))
    removed_swr     = list(filter(lambda word: True if word not in stop_words else False, word_lowered))
    removed_pnc     = list(filter(lambda word: True if word not in punctuation else False, removed_swr))
    freq_in_article = nltk.probability.FreqDist(removed_pnc)
    normalized_freq_in_article = {}
    
    for word, freq in freq_in_article.items():
        normalized_freq_in_article[word] = freq / freq_in_article[freq_in_article.max()]
    
    sent_score = {}
    for sent in sent_lowered:
        for word in nltk.word_tokenize(sent):
            if word in normalized_freq_in_article.keys():
                if sent not in sent_score.keys():
                    sent_score[sent] = normalized_freq_in_article[word]
                else:
                    sent_score[sent] += normalized_freq_in_article[word]

    sent_score = sorted(sent_score.items(), key=lambda x: x[1], reverse=True)
    summed = " ".join([part[0] for part in sent_score[0:2]])
    summarized_content = "".join([summarized_content, "$"+subject+"\n", summed, "\n#\n"])


    print("Summarized atricle regarding {}: ".format(subject))
    print(summed)
    print("Length of orginal text: ", len(article))
    print("Length of summed text: ", len(summed))
    print("=======================================================================================")

with open("summarized.txt", 'w') as file_descriptor:
    file_descriptor.write(summarized_content)

Summarized atricle regarding $Abortion: 
a well functioning health system implies many factors, including: evidence-based policies; universal health coverage; the reliable supply of quality, affordable medical products and equipment; that an adequate number of health workers, of different types, provide abortion care at a reachable distance to patients;  the delivery of abortion care through a variety of approaches, e.g., care in health facilities, digital interventions and self-care approaches, allowing for choices depending on the values and preferences of the pregnant person, available resources, and the national and local context;that health workers are trained to provide safe and respectful abortion care, to support informed decision-making and to interpret laws and policies regulating abortion; that health workers are supported and protected from stigma; andprovision of contraception to prevent unintended pregnancies. availability and accessibility of information implies: provisi